In [ ]:
from os import listdir
from keras.applications.vgg16 import VGG16
from keras.applications.vgg19 import VGG19
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
from keras.models import Model
import cv2
import numpy as np
from google.colab import drive
drive.mount('/content/drive' , force_remount=True)
import pandas as pd
from glob import glob
import os
import sys
from sklearn.preprocessing import MinMaxScaler
sys.path.append('/content/drive/MyDrive/Goodarzi/QuickSelection')
from sklearn.model_selection import train_test_split

In [ ]:
def extractFeatures(input_directory,idx):
    if idx==1:
      # defining model
      model = VGG16()
    else:
      model = VGG19()
    # removing the last layer
    model.layers.pop()
    model = Model(inputs=model.inputs, outputs=model.layers[-1].output)
    print(model.summary())

    # feature extraction from photos
    features = dict()
    for file in os.listdir(input_directory):
        filename = input_directory + '/' + file
        image = load_img(filename, target_size=(224, 224))
        image = img_to_array(image)
        image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
        image = preprocess_input(image)
        features[file.split('.')[0]] = model.predict(image, verbose = 1)

    return features 

In [ ]:
BASE_PATH = '/content/drive/MyDrive/Goodarzi/'
input_directory = BASE_PATH + '/' + 'input'
target_directory = BASE_PATH  + 'target'
feats1 = extractFeatures(input_directory,1)
feats2 = extractFeatures(input_directory,2)

In [ ]:
X_numeric1=list(feats1.values())
X1=np.array(X_numeric1)
Xnew1=X1.reshape(X1.shape[0],X1.shape[2])

X_numeric2=list(feats2.values())
X2=np.array(X_numeric2)
Xnew2=X2.reshape(X2.shape[0],X2.shape[2])
Xnew2.shape
Xnew=np.array([[Xnew1],[Xnew2]])
X=Xnew.reshape(300, 2000)
X.shape

(300, 2000)

In [ ]:
cols = ["a", "b", "c"]
filename = os.listdir(target_directory)
os.chdir(target_directory)
df = pd.read_csv(filename[0] , sep=",", header=None , names=cols)
y = np.asarray(df)
y=np.delete(y, 0 , axis=0)
Y = np.delete(y, [0,2] , axis=1)


xtrain, xtest, ytrain, ytest=train_test_split(X, Y, test_size=0.20)


In [ ]:
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from sklearn import metrics

In [ ]:
#Import Random Forest Model
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, 
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=2000, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

#Create a Gaussian Classifier
clf=RandomForestClassifier(n_estimators=100)

#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(xtrain,ytrain)



In [ ]:
ypred=clf.predict(xtest)
ypred

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

logistic regression

In [ ]:
logreg_clf = LogisticRegression()
logreg_clf.fit(xtrain, ytrain)
ypred_logistic=logreg_clf.predict(xtest)

Support Vector Classifier

In [ ]:
SVC_model = SVC()


K-Nearest Neighbors Classifier

In [ ]:
KNN_model = KNeighborsClassifier(n_neighbors=20)

In [ ]:
SVC_model.fit(xtrain, ytrain)
KNN_model.fit(xtrain, ytrain)

In [ ]:

print("Accuracy:",metrics.accuracy_score(ytest, ypred))